In [943]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import automata.regex.regex as re
from automata.fa.dfa import DFA
# disable warnings
import warnings
warnings.filterwarnings('ignore')

nodes = ['1', '2', '3', '4']
send_edges = [('1', '2', 'a'),('2', '3', 'b'),('3', '4', 'c')]
receive_edges = [('3', '2', 'a'),('4', '3', 'b')]

In [944]:
# take a dfa edges and return the fitting regeular expression
def to_dfa(init, nodes, send_edges, receive_edges, final_states):
    # build transitions out of edges and letters set
    transitions = {}
    letters = set()
    for edge in send_edges:
        letters.add(edge[2])
    # init all transitions to be self referencing
    for node in nodes:
        transitions[node] = {}
        for letter in letters:
            transitions[node][letter] = node
    for edge in receive_edges:
        transitions[edge[0]][edge[2]] = edge[1]
    
    # build DFA
    dfa = DFA(
        states= nodes,
        input_symbols= letters,
        transitions= transitions,
        initial_state=init,
        final_states=final_states
    )   
    # build regex
    return dfa

In [945]:
# create a dfa for each (sending edge) * (nodes) combination
def create_dfa_list(nodes, send_edges, receive_edges):
    dfa_list = {}
    for end_node in nodes:
        for edge in send_edges:
            final_states = {end_node}
            dfa_list[(edge[1], end_node)] = to_dfa(edge[1], set(nodes), send_edges, receive_edges, final_states)
            dfa_list[(edge[0], end_node)] = to_dfa(edge[0], set(nodes), send_edges, receive_edges, final_states)
    return dfa_list

In [946]:
def eval_word(dfa_list, word, init='1'):
    node_sums = {}
    for node in nodes:
        sum = 0
        # print(node)
        for letter_index in range(len(word)):
            letter = word[letter_index]
            # print(letter)
            remaining_word = word[letter_index+1:]
            letter_send_edges = [edge for edge in send_edges if edge[2] == letter]
            for edge in letter_send_edges:
                a1 = dfa_list[(edge[1], node)].accepts_input(remaining_word)
                a2 = dfa_list[(edge[0], node)].accepts_input(remaining_word)
                # if a1 and (node == edge[1]):
                #     a1 = 0
                # if a2 and (node == edge[0]):
                #     a2 = 0
                # print((edge[1], node),(edge[0], node))
                # print(a1, a2)
                sum = sum + a1 - a2
        node_sums[node] = sum
    node_sums[init] = float('inf')
    return node_sums

dfa_list = create_dfa_list(nodes, send_edges, receive_edges)

In [947]:
def send_edge(letter, node_values, send_edges, receive_edges):
    # first, deal with sending edge
    for edge in send_edges:
        # if letter in edge, add to out
        if letter == edge[2]:
            if node_values[edge[0]][0] == 0:
                return 0
            node_values[edge[0]][0] -= 1 # deal with initial not out since sending overrules receiving
            node_values[edge[1]][1] += 1
            break
    # second, deal with receiving edges
    for edge in receive_edges:
        # if letter in edge, add to in
        if letter == edge[2]:
            node_values[edge[0]][2] = node_values[edge[0]][0]
            node_values[edge[1]][1] += node_values[edge[0]][0]
    # commit all in and out
    for node in nodes:
        node_values[node][0] = node_values[node][0] + node_values[node][1] - node_values[node][2]
        node_values[node][1] = 0
        node_values[node][2] = 0
    return 1

# simulate word over the BP
def simulate(word, nodes, send_edges, receive_edges):
    # init node_values
    node_values = {}
    for node in nodes:
        node_values[node] = [0, 0, 0] # [initial, in, out]
    node_values[nodes[0]] = [float('inf'), 0, 0]
    # for each letter in word
    for letter in word:
        # send letter
        if(send_edge(letter, node_values, send_edges, receive_edges) == 0):
            raise Exception('Letter ' + letter + ' has no cores')
    # return node_values first value in each node
    # convert to [(node, node_values[node][0]) for node in nodes] to dict
    return dict([(node, node_values[node][0]) for node in nodes])

In [948]:
def compare_outputs(word, nodes, send_edges, receive_edges):
    # get verifier output
    verifier_output = eval_word(dfa_list, word)
    # get simulate output
    simulate_output = simulate(word, nodes, send_edges, receive_edges)
    # compare outputs
    for node in nodes:
        if verifier_output[node] != simulate_output[node]:
            print('verifier_output', verifier_output)
            print('simulate_output', simulate_output)
            raise Exception(f"Node {node} verifier output {verifier_output[node]} != simulate output {simulate_output[node]}")
    # print('Y')

In [949]:
def calculate_sum_of_letter(word, letter):
    return len([l for l in word if l == letter])

def gen_word_run(word, node_values, send_edges, receive_edges):
    # search for non empty nodes, and rand some number of sending actions from the node
    non_empty_nodes = [node for node in nodes if node_values[node][0] > 0]
    # 
    possible_send_edges = [edge for edge in send_edges if edge[0] in non_empty_nodes]
    all_possible_letters = [edge[2] for edge in possible_send_edges]
    num_of_instances = [calculate_sum_of_letter(word, letter) for letter in all_possible_letters]
    # for each possible letter, give a nuumber
    letter_prob_numerator = [max(sum(num_of_instances) - num_of_instances[i], 1) for i in range(len(all_possible_letters))]
    letter_prob = [letter_prob_numerator[i] / sum(letter_prob_numerator) for i in range(len(letter_prob_numerator))]
    letter = np.random.choice(all_possible_letters, p=letter_prob)
    # get node of letter
    node = [edge[0] for edge in possible_send_edges if edge[2] == letter][0]
    #
    # node = np.random.choice(non_empty_nodes)
    # select number of sending actions, if float('inf') rand up to 10
    max_num_sending_actions = 8 if node_values[node][0] == float('inf') else node_values[node][0]
    # number of sending actions, randomized with favor to lower numbers
    num_sending_actions = np.random.randint(0, max_num_sending_actions)
    # select letters to send
    node_send_edges = [edge[2] for edge in send_edges if edge[0] == node]
    if len(node_send_edges) == 0:
        return ''
    letter = np.random.choice(node_send_edges)
    # send letters
    for i in range(num_sending_actions):
        if(send_edge(letter, node_values, send_edges, receive_edges) == 0):
            break
        word += letter
    return word

def gen_word(send_edges, receive_edges, max_runs):
    # simulate word over the BP
    node_values = {}
    for node in nodes:
        node_values[node] = [0, 0, 0] # [initial, in, out]
    node_values[nodes[0]] = [float('inf'), 0, 0]
    # set random number of runs out of max_runs
    num_of_runs = max_runs#np.random.randint(1, max_runs)
    # run gen_word_run num_of_runs times
    word = ''
    for i in range(num_of_runs):
        word = gen_word_run(word, node_values, send_edges, receive_edges)
    return word


In [950]:
word = 'aaaaaaab'
print(eval_word(dfa_list, word))
compare_outputs(word, nodes, send_edges, receive_edges)

{'1': inf, '2': 6, '3': 1, '4': 0}


In [ ]:
# gen n words and compare outputs
for i in range(100):
    word = gen_word(send_edges, receive_edges, 25)
    print(f"Word {i}: {word}")
    compare_outputs(word, nodes, send_edges, receive_edges)
print('Success!')